# 设备能力查看实验
* 了解 ncclient 的基本用法
* 了解设备能力的概念

In [7]:
import xmltodict
from ncclient import manager

## 建立基础设备信息

In [2]:
iosXE_netconf_info = {"host": "192.168.1.61",
                      "port": 830,
                      "username": "netconf",
                      "password": "netconf123",
                      "hostkey_verify": False}

## 完成设备连接

In [4]:
device = manager.connect(**iosXE_netconf_info)

## 查看设备能力

In [5]:
for cap in device.server_capabilities:
    print(cap)

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:confirmed-commit:1.1
urn:ietf:params:netconf:capability:confirmed-commit:1.0
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:xpath:1.0
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=explicit&also-supported=report-all-tagged,report-all
urn:ietf:params:netconf:capability:yang-library:1.0?revision=2016-06-21&module-set-id=b55ee7d4a52850f3f62eb64ef0a87966
http://tail-f.com/ns/netconf/actions/1.0
http://cisco.com/ns/cisco-xe-ietf-ip-deviation?module=cisco-xe-ietf-ip-deviation&revision=2016-08-10
http://cisco.com/ns/cisco-xe-ietf-ipv4-unicast-routing-deviation?module=cisco-xe-ietf-ipv4-unic

# 获取全部配置后，过滤出用户信息

In [9]:
with manager.connect(**iosXE_netconf_info) as device:
    full_config = device.get_config(source="running")
    # 将数据变为原始的XML数据
    xml_data = full_config.xml

# 将 xml_data 里面的设备数据解析为 Python 的 Orderdict 数据类型
# py_data 就是适用于 python 的全部配置
py_data = xmltodict.parse(full_config.xml)

# 我们进一步从中取出用户配置信息，并将这些信息合理输出
user_data = py_data["rpc-reply"]["data"]["native"]["username"]
username = user_data["name"]
secret = user_data["secret"]["secret"]
privilege = user_data["privilege"]
print(f"用户名：{username}\n密文密码：{secret}\n用户权限级别：{privilege}")

用户名：netconf
密文密码：$9$Q.xe6b0dLRL5XE$ONVOgI8Gnowugef7bA3Po5qbnz6Hf0o0nN1h94Z7s4c
用户权限级别：15


# 使用XML的过滤仅获取用户信息

In [12]:
from xml.dom.minidom import parseString as xmlpar

netconf_conf_filter = '''
<filter xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
    <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
        <username>
        </username>
    </native>
</filter>
'''

with manager.connect(**iosXE_netconf_info) as device:
    # 使用 get_config 方法，向 running 数据库做查询
    full_config = device.get_config(source="running",filter=netconf_conf_filter)
    xml_data = full_config.xml
    print(xmlpar(xml_data).toprettyxml(indent='  '))

<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:a9eb1445-d879-4b40-8196-2bf5a67e047e" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <data>
    <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
      <username>
        <name>netconf</name>
        <privilege>15</privilege>
        <secret>
          <encryption>9</encryption>
          <secret>$9$Q.xe6b0dLRL5XE$ONVOgI8Gnowugef7bA3Po5qbnz6Hf0o0nN1h94Z7s4c</secret>
        </secret>
      </username>
    </native>
  </data>
</rpc-reply>



# 完成hostname的配置

In [13]:
# 设定一个想配置上去的 Hostname
hostname = "myIOSXE"

# Hostname 的配置数据
hostname_data = f"""
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
     <hostname>{hostname}</hostname>
  </native>
</config>
"""

# Hostname 的查询过滤器
hostname_filter = """
<filter xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
    <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
        <hostname>
        </hostname>
    </native>
</filter>
"""

# 完成配置
with manager.connect(**iosXE_netconf_info) as device:
    hostname_result = device.edit_config(target="candidate", config=hostname_data)
    # 通过 commit 方法可以将配置同步到 running 数据库
    device.commit()
    print("Hostname修改完成")
    
print("目前的Hostname配置是：")

# 完成查询
with manager.connect(**iosXE_netconf_info) as device:
    hostname_config = device.get_config(source="running",filter=hostname_filter)
    xml_data = hostname_config.xml
    py_data = xmltodict.parse(xml_data)
    new_hostname = py_data['rpc-reply']['data']['native']['hostname']
    print(new_hostname)

Hostname修改完成
目前的Hostname配置是：
myIOSXE


# 环回接口的增删改查可以查看 project 文件夹中的项目